In [16]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# Connect to the database
def extract_tables(table_list=None):
    conn = sqlite3.connect('database.sqlite')
    c = conn.cursor()
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = c.fetchall()
    df_dict = dict()

    for table in tables:
        df_name = f"df_{table[0]}"
        #print(df_name)
        if table_list:
            if df_name in table_list:
                print(f"Extracting table: {table[0]}")
                df_dict.update({df_name:pd.read_sql_query(f"SELECT * from {table[0]}", conn)})
        else:
            print('no table list')
            df_dict.update({df_name: pd.read_sql_query(f"SELECT * from {table[0]}", conn)})
            
    conn.close()

    return df_dict

df_dict = extract_tables(table_list=['df_Player_Attributes', 'df_Match'])

Extracting table: Player_Attributes
Extracting table: Match


In [17]:
def clean_up_players_df(df_dict):
    '''make necessary changes to the player attributes dataframe for further processing and analysis'''
    df_players_attributes = df_dict['df_Player_Attributes']
    #convert to datetime object
    df_players_attributes['date'] = pd.to_datetime(df_players_attributes['date'])

    #add dummy variables for categorical columns
    for col in df_players_attributes.columns:
        if len(df_players_attributes[col].unique()) < 25:
            df_players_attributes = pd.concat([df_players_attributes, pd.get_dummies(df_players_attributes[col], prefix=col)], axis=1)
            df_players_attributes.drop(col, axis=1, inplace=True)
        
        #scale numerical columns
        elif col not in ['id', 'player_fifa_api_id', 'player_api_id', 'date']:
            df_players_attributes[col] = (df_players_attributes[col] - df_players_attributes[col].mean()) / df_players_attributes[col].std()
    
    return df_players_attributes

df_players_attributes = clean_up_players_df(df_dict)
df_players_attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,preferred_foot_left,preferred_foot_right,attacking_work_rate_None,attacking_work_rate_high,attacking_work_rate_le,attacking_work_rate_low,attacking_work_rate_medium,attacking_work_rate_norm,attacking_work_rate_stoc,attacking_work_rate_y,defensive_work_rate_0,defensive_work_rate_1,defensive_work_rate_2,defensive_work_rate_3,defensive_work_rate_4,defensive_work_rate_5,defensive_work_rate_6,defensive_work_rate_7,defensive_work_rate_8,defensive_work_rate_9,defensive_work_rate__0,defensive_work_rate_ean,defensive_work_rate_es,defensive_work_rate_high,defensive_work_rate_low,defensive_work_rate_medium,defensive_work_rate_o,defensive_work_rate_ormal,defensive_work_rate_tocky
0,1,218353,505942,2016-02-18,-0.227238,-0.373218,-0.353024,-0.311002,0.832922,-0.100723,-0.299532,-0.460710,-0.436337,-0.582161,0.481443,-0.946844,-0.589938,-0.322302,-0.538104,-2.086603,-0.014506,-0.421963,-0.814869,-0.990375,0.710344,-0.998498,0.624752,0.924967,-0.584689,-0.255780,-0.450518,0.858679,0.868041,0.972210,-0.516107,-0.319121,-0.512673,-0.505129,-0.490834,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2,218353,505942,2015-11-19,-0.227238,-0.373218,-0.353024,-0.311002,0.832922,-0.100723,-0.299532,-0.460710,-0.436337,-0.582161,0.481443,-0.946844,-0.589938,-0.322302,-0.538104,-2.086603,-0.014506,-0.421963,-0.814869,-0.990375,0.710344,-0.998498,0.624752,0.924967,-0.584689,-0.255780,-0.450518,0.858679,0.868041,0.972210,-0.516107,-0.319121,-0.512673,-0.505129,-0.490834,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,3,218353,505942,2015-09-21,-0.937350,-1.131682,-0.353024,-0.311002,0.832922,-0.100723,-0.299532,-0.460710,-0.436337,-0.582161,0.481443,-0.946844,-0.589938,-0.322302,-0.538104,-2.086603,-0.014506,-0.421963,-0.814869,-0.990375,0.710344,-0.998498,0.127534,-0.566026,-0.584689,-0.255780,-0.450518,0.858679,0.728401,0.972210,-0.516107,-0.319121,-0.512673,-0.505129,-0.490834,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,4,218353,505942,2015-03-20,-1.079373,-1.283375,-0.411021,-0.363527,0.772275,-0.171175,-0.354306,-0.517065,-0.491114,-0.638241,0.411972,-1.012648,-0.589938,-0.322302,-0.538104,-2.195829,-0.014506,-0.483939,-0.814869,-0.990375,0.710344,-1.052943,0.065381,-0.617439,-0.638894,-0.321812,-0.514841,0.717354,0.588760,0.833313,-0.575400,-0.382143,-0.559286,-0.567244,-0.548980,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,5,218353,505942,2007-02-22,-1.079373,-1.283375,-0.411021,-0.363527,0.772275,-0.171175,-0.354306,-0.517065,-0.491114,-0.638241,0.411972,-1.012648,-0.589938,-0.322302,-0.538104,-2.195829,-0.014506,-0.483939,-0.814869,-0.990375,0.710344,-1.052943,0.065381,-0.617439,-0.638894,-0.321812,-0.514841,0.717354,0.588760,0.833313,-0.575400,-0.382143,-0.559286,-0.567244,-0.548980,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [21]:
def clean_up_matches_df(df_dict):
    '''make necessary adjustments to the matches dataframe for further processing and analysis such as dropping empty columns or rows, and adding in a column for home team result
    Note that home_player_1	etc are player ids for the given teams and can be used to join with the player attributes dataframe'''

    df_matches = df_dict['df_Match']

    #convert date to datetime object
    df_matches['date'] = pd.to_datetime(df_matches['date'])

    total = df_matches.shape[0]
    for col in df_matches.columns:
        #drop columns with greater than 10% missing values
        if df_matches[col].isnull().sum()/total > 0.1:
            df_matches.drop(col, axis=1, inplace=True)
        elif col in ['goal', 'shoton', 'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession','country_id', 'league_id','stage','season']:
            df_matches.drop(col, axis=1, inplace=True)
        
    #drop rows with missing values
    df_matches.dropna(axis=0, inplace=True)

    #add column for home team win, draw, away team win
    df_matches['home_win'] = np.where(df_matches['home_team_goal'] > df_matches['away_team_goal'], 1, 0)
    df_matches['home_draw'] = np.where(df_matches['home_team_goal'] == df_matches['away_team_goal'], 1, 0)
    df_matches['home_loss'] = np.where(df_matches['home_team_goal'] < df_matches['away_team_goal'], 1, 0)

    
    return df_matches


df_matches = clean_up_matches_df(df_dict)
print(df_matches.shape)
df_matches.head()

(21361, 76)


,id,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,home_win,home_draw,home_loss
145,146,2009-02-27,493017,8203,9987,2,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,38327.0,67950.0,67958.0,67959.0,37112.0,36393.0,148286.0,67898.0,164352.0,38801.0,26502.0,37937.0,38293.0,148313.0,104411.0,148314.0,37202.0,43158.0,9307.0,42153.0,32690.0,38782.0,1,0,0
153,154,2009-03-08,493025,9984,8342,1,3,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,36835.0,37047.0,37021.0,37051.0,104386.0,32863.0,37957.0,37909.0,38357.0,37065.0,78462.0,37990.0,21812.0,11736.0,37858.0,38366.0,37983.0,39578.0,38336.0,52280.0,27423.0,38440.0,0,0,1
155,156,2009-03-07,493027,8635,10000,2,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,34480.0,38388.0,26458.0,13423.0,38389.0,30949.0,38393.0,38253.0,38383.0,38778.0,37069.0,37900.0,37886.0,37903.0,37889.0,94030.0,37893.0,37981.0,131531.0,130027.0,38231.0,131530.0,1,0,0
162,163,2009-03-13,493034,8203,8635,2,1,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,38327.0,67950.0,67958.0,38801.0,67898.0,37112.0,67959.0,148286.0,164352.0,33657.0,26502.0,34480.0,38388.0,38389.0,31316.0,164694.0,30949.0,38378.0,38383.0,38393.0,38253.0,37069.0,1,0,0
168,169,2009-03-14,493040,10000,9999,0,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,37900.0,37886.0,37100.0,37903.0,37889.0,37893.0,37981.0,131531.0,131530.0,38231.0,130027.0,38318.0,38247.0,16387.0,94288.0,94284.0,45832.0,26669.0,33671.0,163670.0,37945.0,33622.0,0,1,0
